In [0]:
pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

In [0]:
import lyricsgenius
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
genius = lyricsgenius.Genius("iihP9u6cVKE79Xtu99OH6rVZlMUzIMeOvIVs8BDQNDeVmZhXJXEaI28E6QOc4BdB")
genius.remove_section_headers = True

In [0]:
artist = genius.search_artist("Oxxxymiron", max_songs=50, sort="title")

Searching for songs by Oxxxymiron...

Song 1: "16 Bars Acapella"
Song 2: "Afterparty (Demo)"
Song 3: "AI Ogon"
Song 4: "Amplify and simplify (Freestyle)"
Song 5: "CCTV"
Song 6: "Darkside"
Song 7: "Freestyle #1"
Song 8: "Ganz Promo Tune"
Song 9: "Hangover"
Song 10: "HPL"
Song 11: "IMPERIVM"
Song 12: "Intro"
Song 13: "OXXXYMIRON"
Song 14: "Russky Cockney"
Song 15: "Shade 45 Freestyle (Идея)"
Song 16: "Street Freestyle battle"
Song 17: "Ultima Thule"
Song 18: "Unreleased Track"
Song 19: "Unreleased Track 2"
Song 20: "Unreleased Track 3"
Song 21: "Unreleased Track 4"
Song 22: "XXX SHOP"
Song 23: "Башня из слоновой кости (Ivory Tower)"
Song 24: "Биполярочка (Bipolarochka)"
Song 25: "Больше Бена (Bigga Than Ben)"
Song 26: "В бульбуляторе (In the Bong)"
Song 27: "В говне (In Shit)"
Song 28: "В долгий путь (1 раунд 17ib) (On a Long Journey)"
Song 29: "Ветер перемен (2 раунд 17ib) (The Wind of Change)"
Song 30: "Вечный жид (Everlasting Jew)"
Song 31: "Витязи словоблудия (Уховертка) (The Knights

In [0]:
artist.save_lyrics(extension='txt')

Wrote `Lyrics_Oxxxymiron.txt`


In [0]:
lyricss = open("Lyrics_Oxxxymiron.txt","r")

In [0]:
song4 = []
for line in lyricss:
  song4.append(word_tokenize(line, ))

In [0]:
song7 = []
for i in song4:
  stroke = []
  for j in i:
    stroke.append(j.lower())
  song7.append(stroke)


In [0]:
for strokes in song7:
  strokes.append('\n')

In [0]:
song7

In [0]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [0]:
from nltk.lm import MLE
model = MLE(2)

In [0]:
from nltk.lm import KneserNeyInterpolated

In [0]:
model2 = KneserNeyInterpolated(2)

In [0]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train3, vocabul3 = padded_everygram_pipeline(2, song7)

In [0]:
len(model.vocab)

0

In [0]:
model.fit(train3, vocabul3)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 8376 items>


In [0]:
model2.fit(train, vocabul)
print(model2.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 8376 items>


In [0]:
print(model2.counts)

<NgramCounter with 2 ngram orders and 66527 ngrams>


In [0]:
print(model.generate(20,random_seed = 41))

['I', 'don', '’', 'm', 'at', 'my', 'home', 'when', 'I', 'don', '’', 'm', 'at', 'my', 'home', 'when', 'I', 'don', '’', 'm']


In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token != '</s>':
            content.append(token)
    return detokenize(content)

In [0]:
print(generate_sent(model, 200, random_seed = 41))

<UNK>, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав, едва заслышав ,


In [0]:
artist = genius.search_artist("Oxxxymiron", max_songs=3, sort="title")

Searching for songs by Oxxxymiron...

Song 1: "16 Bars Acapella"
Song 2: "Afterparty (Demo)"
Song 3: "AI Ogon"

Reached user-specified song limit (3).
Done. Found 3 songs.


In [0]:
print(genius.search_song("16 Bars Acapella", "Oxxxymiron").lyrics)

In [0]:
print(artist.songs)

[('Alexander All Alone', 'Andy Shauf'), ('All the Same', 'Andy Shauf'), ('Angela', 'Andy Shauf'), ('To You', 'Andy Shauf')]
